In [1]:
import random
import requests
import json
import ssl
import websocket

### Задание 1
Посмотреть документацию к API GitHub, разобраться как вывести список репозиториев для конкретного пользователя,  
сохранить JSON-вывод в файле *.json.

Задаем токен авторизации и имя интересующего пользователя:

In [74]:
MY_TOKEN = ''
user = 'KirillKras'

Создаем объект сессии с указанным токеном авторизации

In [49]:
session = requests.Session()
session.headers['Authorization'] = f'bearer {MY_TOKEN}'

Так как, согласно схеме валидации, запрос GraphQL API на получение репозиториев пользователя должен содержать информацию  
о количестве запрашиваемых репозиториев, сначала формируем запрос на получение количества репозитриев: 

In [101]:
query_count = '''
{
    user(login: "%s"){
        repositories {
            totalCount
        }
    }
}
''' % (user)

Выполняем запрос типа Query через объект сессии:

In [109]:
response = session.post('https://api.github.com/graphql', json={'query': query_count})

В полученном ответе, есои он был успешным, получаем количество репозиториев:

In [110]:
if response.ok:
    json_count = response.json()
    total_count = json_count['data']['user']['repositories']['totalCount']
print(f'У пользователя {user} количество репозиториев - {total_count}')

У пользователя KirillKras количество репозиториев - 11


Формируем запрос на получение репозиториев по имени пользователя и ранее установленному количеству,  
получаем имя репозитория, его краткое описание и реального владельца репозитория если форк:

In [105]:
query_repos = '''
{
    user(login: "%s"){
        repositories (first: %s) {
            edges {
                node {
                    name
                    description
                    parent {
                        owner {
                            login
                        }
                    }
                }
            }
        }
    }
}
''' % (user, total_count)

Выполняем запрос типа Query через объект сессии:

In [112]:
response = session.post('https://api.github.com/graphql', json={'query': query_repos})

Получаем ответ и выводим результаты в зависимости от наличия запрошенных полей: 

In [113]:
print(f'Список репозиториев пользователя {user}:')
json_repos = response.json()
for edge in json_repos['data']['user']['repositories']['edges']:
    result = f"Репозиторий: {edge['node']['name']} "
    if edge['node']['description']:
        result += f"\n\tкраткое описание: {edge['node']['description']}"
    if edge['node']['parent']:
        result += f"\n\tэто форк, реальный владелец {edge['node']['parent']['owner']['login']}"
    print(result)

Список репозиториев пользователя KirillKras:
Репозиторий: imagestore 
	краткое описание: Django gallery solution.
	это форк, реальный владелец hovel
Репозиторий: git-repo 
Репозиторий: alg_les1 
	краткое описание: Курс алгоритмы, лекция 1
Репозиторий: algs_structs 
	краткое описание: Курс "Алгоритмы и структуры данных"
Репозиторий: python_basic 
Репозиторий: Python_DS 
	краткое описание: Homework for "Python Data Science"
Репозиторий: in_math 
	краткое описание: курс "Введение в высшую математику"
Репозиторий: Linux 
	краткое описание: Курс "Linux. Рабочая станция"
Репозиторий: matan 
	краткое описание: Введение в математический анализ
Репозиторий: TerVer 
	краткое описание: Курс "Теория вероятности и мат. статистика"
Репозиторий: Parsing 


Сохраняем ответ в файл в формате json:

In [126]:
with open(f'{user}_repos.json', 'w') as f:
    f.write(json.dumps(['query_repos', json_repos], indent=4))

### Задание 2
Изучить список открытых API. Найти среди них любое, требующее авторизацию (любого типа).  
Выполнить запросы к нему, пройдя авторизацию. Ответ сервера записать в файл.

Решаемая задача - получение потока новых постов из ВК через VK Stream API  
с фильтром по ключевому слову "Москва"

В настройках ВК создаем приложение, получаем сервисный ключ, запускаем его:

In [ ]:
vk_token = '...'

По ключу получаем от ВК сервер и токен потока:

In [2]:
url = f'https://api.vk.com/method/streaming.getServerUrl?v=5.64&access_token={vk_token}'
response = requests.get(url).json()
endpoint, key = [response["response"]["endpoint"], response["response"]["key"]]
print(f'EndPoint: {endpoint},\nKey: {key}')

EndPoint: streaming.vk.com,
Key: 960dbc354c7d4fdeda4c1193c358233ab1896a16


Создаем и добавляем правило - фильтр постов по ключевому слову "Москва"  
и сдучайной меткой правила в диапазоне (11111, 99999)

In [5]:
value = 'Москва'
rule_params = {
    'rule': {
        'value': value, 
        'tag': 'tag_' + str(random.randint(11111, 99999))
    }
}
headers = {
    'content-type': 'application/json'
}
response_json = requests.post(f'https://{endpoint}/rules?key={key}', 
                              data=json.dumps(rule_params), headers=headers).json()
print(f'Добавление правила {response_json["code"] == 200}')

Добавление правила True


Создание обработчиков событий websocket:

In [6]:
def on_message(ws, message):
    print(f'Получено сообщение: {message}')
    
def on_error(ws, error):
    print(f'Получена ошибка: {error}')
    
def on_open(ws):
    print('Поток открыт')
    
def on_close(ws):
    print('Поток закрыт')

Создание объекта websocket по ранее полученным серверу и токену потока, обработчикам событий. 
На ВК Streaming API работает только по WSS, поэтому необходима передача параметров SSL.  
Запускаем websocket получаем сообщения по заданному фильтру

In [7]:
    
websocket.enableTrace(True)
ws = websocket.WebSocketApp(f'wss://{endpoint}/stream?key={key}',                             
                            on_message=on_message,
                            on_error=on_error,
                            on_open=on_open,
                            )
ws.on_close=on_close
ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})

--- request header ---
GET /stream?key=960dbc354c7d4fdeda4c1193c358233ab1896a16 HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: streaming.vk.com
Origin: http://streaming.vk.com
Sec-WebSocket-Key: VtIwCuVAcqdxPPFa3vXVRA==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Server: nginx/1.12.1
Date: Mon, 16 Dec 2019 12:17:56 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: 2NYEPSGPtzzALV+QDn/n18EZJ98=
-----------------------
send: b'\x8a\x80\xb5~\x92\x95'
send: b'\x8a\x80\x9e\xdd\xf3\xb1'
send: b'\x8a\x80\xaf\x19\xdbq'
send: b'\x8a\x80\xc5Fb*'
send: b'\x8a\x808?\xf3\xf5'
send: b'\x88\x82\x93l\xbf\x12\x90\x84'


Поток открыт
Получено сообщение: {"code":100,"event":{"action":"new","action_time":1576498731,"author":{"author_url":"https://vk.com/id445794643","id":445794643,"platform":7},"creation_time":1576498731,"event_id":{"topic_owner_id":-49815148,"topic_id":34678244,"topic_post_id":134849},"event_type":"topic_post","event_url":"https://vk.com/topic-49815148_34678244?post=134849","tags":["tag_19797","tag_56038","tag_73345","tag_78713"],"text":"🎄🎁🎄 Пассажирские перевозки в ✔КРЫМ✔РОССИЮ✔АБХАЗИЮ✔УКРАИНУ✔ГРУЗИЮ✔АРМЕНИЮ✔БЕЛАРУСЬ регулярно на комфортабельных автобусах из Донецка и области, бронируем места на обратную дорогу. Детям - скидки. Только официальные перевозчики&#33; Только опытные водители&#33; ☎ 071-335-00-22 + Viber. Водафон 095-353-37-34. 🎄🎁🎄\u003cbr\u003eСУПЕРЦЕНА&#33; КРЫМ - от 1600 руб. РОСТОВ - от 550 руб. МОСКВА, ТУЛА - от 1300 руб. \u003cbr\u003e🇷🇺-МОСКВА - от 1300 руб. \u003cbr\u003e🇷🇺-ВОРОНЕЖ - 1200 руб. \u003cbr\u003e🇷🇺-ТУЛА - от 1300 руб. \u003cbr\u003e\u003cbr\u003e🇷🇺-Таганр